# Dynamic Programming for Reinforcement Learning with Gridworld Examples

### This notebook was created to practice applying dynamic programming to reinforcement learning. The notes were mainly intended to help me reflect on what I have learned. If you somehow find this, welcome, and please feel free to contact me at joeynaro@rocketmail.com

In [1]:
from grid_world_env_deterministic import grid_world_env_deterministic as deterministic_enviro
from grid_renderer import grid_renderer
import turtle
import numpy as np
import random

np.set_printoptions(suppress=True)#precision=4)

I have created a few simple gridworld environments. Below I have used one of these environments as the gridworld I am going to solve. The gridworld environments are in my GitHub repo if you are interested in trying them out.

In [2]:
height = 5
width = 5
hole_id = 4
goal_id = 3
num_of_acts = 4
terminal_state = 5
test = deterministic_enviro()
state, _, _, _ = test.reset()
state = state[0:int(len(state)/2)]
state = np.reshape(state, (height, width))

This is what the environment looks like. The black represent holes, and the red represents the goal
![image of environment](./images/blank_environment.png)

For the first attempt, I have greatly penalized falling in the holes and set the goal to always be zero. Let's see how that goes.

In [3]:
rewards = np.zeros((height, width))
hole_locations = state == hole_id
goal_location = state == goal_id

hole_penalty = -10
rewards[hole_locations] = hole_penalty
print(rewards)

[[  0.   0.   0. -10.   0.]
 [-10.   0.   0.   0.   0.]
 [  0. -10.   0.   0. -10.]
 [  0.   0.   0.   0.   0.]
 [  0.   0.   0. -10.   0.]]


The following defines the state that will follow each state given an action. This environment is deterministic.

In [4]:
actions = []
for start_row in range(height):
    actions.append([])
    for start_col in range(width):
        actions[start_row].append([])
        for action in range(num_of_acts):
            row = start_row
            col = start_col
            if action == 0 and row > 0:
                row -= 1
            elif action == 1 and col < width - 1:
                col += 1
            elif action == 2 and row < height - 1:
                row += 1
            elif action == 3 and col > 0:
                col -= 1
            actions[start_row][start_col].append((col, row))

The starting policy will find each action equally likely, except in the terminal goal state, which there is no probability of leaving.

In [5]:
policy = np.zeros((height, width, num_of_acts))
for row in range(height):
    for col in range(width):
        if not goal_location[row][col]:
            for action in range(num_of_acts):
                policy[row][col][action] = 1.0 / num_of_acts
                
print(policy)

[[[0.25 0.25 0.25 0.25]
  [0.25 0.25 0.25 0.25]
  [0.25 0.25 0.25 0.25]
  [0.25 0.25 0.25 0.25]
  [0.25 0.25 0.25 0.25]]

 [[0.25 0.25 0.25 0.25]
  [0.25 0.25 0.25 0.25]
  [0.25 0.25 0.25 0.25]
  [0.25 0.25 0.25 0.25]
  [0.25 0.25 0.25 0.25]]

 [[0.25 0.25 0.25 0.25]
  [0.25 0.25 0.25 0.25]
  [0.25 0.25 0.25 0.25]
  [0.25 0.25 0.25 0.25]
  [0.25 0.25 0.25 0.25]]

 [[0.25 0.25 0.25 0.25]
  [0.25 0.25 0.25 0.25]
  [0.25 0.25 0.25 0.25]
  [0.25 0.25 0.25 0.25]
  [0.25 0.25 0.25 0.25]]

 [[0.25 0.25 0.25 0.25]
  [0.25 0.25 0.25 0.25]
  [0.25 0.25 0.25 0.25]
  [0.25 0.25 0.25 0.25]
  [0.   0.   0.   0.  ]]]


These are the parameters I will use to start with

In [6]:
gamma = 0.9
theta = 0.1

The following is an implementation of synchronous value iteration

In [7]:
delta = 1.0

#Values are intialized arbitrarily 
values = np.zeros((height, width))
values += -0.5
values[hole_locations] = hole_penalty
values[goal_location] = 0
old_values = np.array(values)

synchronous_step = 1
while delta > theta:
    for row in range(height):
        for col in range(width):
            if not goal_location[row][col]:
                next_state_vals = []
                for action in range(num_of_acts):
                    next_spot = actions[row][col][action]
                    if not (next_spot[0] == col and next_spot[1] == row):
                        next_val = rewards[row][col] + (gamma * old_values[next_spot[1]][next_spot[0]])
                        next_state_vals.append(next_val)
                if len(next_state_vals) != 0:
                    values[row][col] = max(next_state_vals)
    
    delta = np.amax(abs(np.subtract(old_values, values)))
    print(delta)
    print('step ', synchronous_step)
    synchronous_step += 1
    
    old_values = np.array(values)
    values = np.array(rewards)

    print(old_values)
    
values = old_values

0.5
step  1
[[ -0.45  -0.45  -0.45 -10.45  -0.45]
 [-10.45  -0.45  -0.45  -0.45  -0.45]
 [ -0.45 -10.45  -0.45  -0.45 -10.45]
 [ -0.45  -0.45  -0.45  -0.45   0.  ]
 [ -0.45  -0.45  -0.45 -10.     0.  ]]
0.45
step  2
[[ -0.405  -0.405  -0.405 -10.405  -0.405]
 [-10.405  -0.405  -0.405  -0.405  -0.405]
 [ -0.405 -10.405  -0.405  -0.405 -10.   ]
 [ -0.405  -0.405  -0.405   0.      0.   ]
 [ -0.405  -0.405  -0.405 -10.      0.   ]]
0.405
step  3
[[ -0.3645  -0.3645  -0.3645 -10.3645  -0.3645]
 [-10.3645  -0.3645  -0.3645  -0.3645  -0.3645]
 [ -0.3645 -10.3645  -0.3645   0.     -10.    ]
 [ -0.3645  -0.3645   0.       0.       0.    ]
 [ -0.3645  -0.3645  -0.3645 -10.       0.    ]]
0.36450000000000005
step  4
[[ -0.32805  -0.32805  -0.32805 -10.32805  -0.32805]
 [-10.32805  -0.32805  -0.32805   0.       -0.32805]
 [ -0.32805 -10.32805   0.        0.      -10.     ]
 [ -0.32805   0.        0.        0.        0.     ]
 [ -0.32805  -0.32805   0.      -10.        0.     ]]
0.32805000000000006

These results, while not great, are certainly interesting. Because of the values I provided, the algorithm has calculated that it is good to reach the goal, but everywhere else is similarly pleasant so long as the agent is not falling into a hole. You can also see where the changes that would happen outside of the bottom right fall below the tolerance, theta.

I can check these numbers with linear algebra, though that would be prohibitively resource expensive on a larger scale. The other problem is that the matrix, 1 - the policy matrix, must be invertible. Let's see if that's the case.

In [8]:
lin_al_rewards = np.ndarray.flatten(rewards)
lin_al_policy = np.zeros((height * width, height * width))

start_index = 0
for row in range(height):
    for col in range(width):
        if (not hole_locations[row][col]) and (not goal_location[row][col]):
            for next_spot in actions[row][col]:
                next_index = (width * next_spot[1]) + next_spot[0]
                lin_al_policy[start_index][next_index] += (1.0 / num_of_acts)
        start_index += 1

print('Rank: ', np.linalg.matrix_rank(1 - lin_al_policy))
if (lin_al_policy.shape[0] == lin_al_policy.shape[1]) and (np.linalg.matrix_rank(1 - lin_al_policy) == (height * width)):
    lin_al_values = np.matmul(np.linalg.inv(1 - (gamma * lin_al_policy)), lin_al_returns)
    print(lin_al_values)
else:
    print('Matrix is singular')

Rank:  20
Matrix is singular


While disappointing, it this not unexpected, because of the environment's states. Let's try something new.

It would appear that in my first attempt, the agent went to the safety of the goal to avoid being near the holes in most cases. However, the agent calculated that the best thing to do in the lower left was to stay there indefinitely. To penalize this, I want to add a penalty for every step taken.

In [9]:
step_penalty = -1

for row in range(height):
    for col in range(width):
        if (not hole_locations[row][col]) and (not goal_location[row][col]):
            rewards[row][col] = step_penalty

print(rewards)

[[ -1.  -1.  -1. -10.  -1.]
 [-10.  -1.  -1.  -1.  -1.]
 [ -1. -10.  -1.  -1. -10.]
 [ -1.  -1.  -1.  -1.  -1.]
 [ -1.  -1.  -1. -10.   0.]]


In [10]:
delta = 1.0

#Values are intialized arbitrarily 
values = np.zeros((height, width))
values += -0.5
values[hole_locations] = hole_penalty
values[goal_location] = 0
old_values = np.array(values)

synchronous_step = 1
while delta > theta:
    for row in range(height):
        for col in range(width):
            if not goal_location[row][col]:
                next_state_vals = []
                for action in range(num_of_acts):
                    next_spot = actions[row][col][action]
                    if not (next_spot[0] == col and next_spot[1] == row):
                        next_val = rewards[row][col] + (gamma * old_values[next_spot[1]][next_spot[0]])
                        next_state_vals.append(next_val)
                if len(next_state_vals) != 0:
                    values[row][col] = max(next_state_vals)
    
    delta = np.amax(abs(np.subtract(old_values, values)))
    print(delta)
    print('step ', synchronous_step)
    synchronous_step += 1
    
    old_values = np.copy(values)
    values = np.copy(rewards)

    print(old_values)
    
values = old_values

0.95
step  1
[[ -1.45  -1.45  -1.45 -10.45  -1.45]
 [-10.45  -1.45  -1.45  -1.45  -1.45]
 [ -1.45 -10.45  -1.45  -1.45 -10.45]
 [ -1.45  -1.45  -1.45  -1.45  -1.  ]
 [ -1.45  -1.45  -1.45 -10.     0.  ]]
0.8550000000000004
step  2
[[ -2.305  -2.305  -2.305 -11.305  -2.305]
 [-11.305  -2.305  -2.305  -2.305  -2.305]
 [ -2.305 -11.305  -2.305  -2.305 -10.9  ]
 [ -2.305  -2.305  -2.305  -1.9    -1.   ]
 [ -2.305  -2.305  -2.305 -10.      0.   ]]
0.7695000000000007
step  3
[[ -3.0745  -3.0745  -3.0745 -12.0745  -3.0745]
 [-12.0745  -3.0745  -3.0745  -3.0745  -3.0745]
 [ -3.0745 -12.0745  -3.0745  -2.71   -10.9   ]
 [ -3.0745  -3.0745  -2.71    -1.9     -1.    ]
 [ -3.0745  -3.0745  -3.0745 -10.       0.    ]]
0.6925500000000007
step  4
[[ -3.76705  -3.76705  -3.76705 -12.76705  -3.76705]
 [-12.76705  -3.76705  -3.76705  -3.439    -3.76705]
 [ -3.76705 -12.76705  -3.439    -2.71    -10.9    ]
 [ -3.76705  -3.439    -2.71     -1.9      -1.     ]
 [ -3.76705  -3.76705  -3.439   -10.        0.

It looks like success, but let's check the policy. The code below gives a greedy policy correlating to the values calculated above. For this policy, and all of my gridworld environments, 0 is up, 1 is right, 2 is down, and 3 is left. The 5's below represent terminal states. Here is a graphical interpretation of it from my gridworld renderer.

In [11]:
#TODO: put this in a separate file

new_policy = np.zeros((height, width))

index = 0
for row in range(height):
    for col in range(width):
        if (not hole_locations[row][col]) and (not goal_location[row][col]):
            max_next_state_val = -100000 #this should never naturally occur
            for action in range(num_of_acts):
                if (actions[row][col][action][0] != col or actions[row][col][action][1] != row):
                    next_spot = actions[row][col][action]
                    next_state_val = values[next_spot[1]][next_spot[0]]
                    if next_state_val > max_next_state_val:
                        max_next_state_val = next_state_val
                        new_policy[row][col] = action
        else:
            new_policy[row][col] = terminal_state

        index += 1
        
print(new_policy)

[[1. 1. 2. 5. 2.]
 [5. 1. 1. 2. 3.]
 [2. 5. 1. 2. 5.]
 [1. 1. 1. 1. 2.]
 [0. 0. 0. 5. 5.]]


That is a resounding success. Here's a visual representation of the policy.
#TODO. Add policy visual

Thus far, I've been using synchronous value iteration. I want to see if asynchronous value iteration can do the job in fewer iterations.

In [12]:
delta = 1.0

#Values are intialized arbitrarily 
values = np.zeros((height, width))
values += -0.5
values[hole_locations] = hole_penalty
values[goal_location] = 0
old_values = np.copy(values)

asynchronous_step = 1
while delta > theta:
    for row in range(height):
        for col in range(width):
            if not goal_location[row][col]:
                next_state_vals = []
                for action in range(num_of_acts):
                    next_spot = actions[row][col][action]
                    if not (next_spot[0] == col and next_spot[1] == row):
                        next_val = rewards[row][col] + (gamma * values[next_spot[1]][next_spot[0]])
                        next_state_vals.append(next_val)
                if len(next_state_vals) != 0:
                    values[row][col] = max(next_state_vals)
    
    delta = np.amax(abs(np.subtract(old_values, values)))
    print(delta)
    print('step ', asynchronous_step)
    asynchronous_step += 1
    
    old_values = np.copy(values)

    print(values)

1.8049999999999997
step  1
[[ -1.45   -1.45   -1.45  -10.45   -1.45 ]
 [-10.45   -1.45   -1.45   -1.45   -2.305]
 [ -1.45  -10.45   -1.45   -1.45  -10.45 ]
 [ -1.45   -1.45   -1.45   -1.45   -1.   ]
 [ -1.45   -1.45   -2.305 -10.      0.   ]]
1.6245
step  2
[[ -2.305   -2.305   -2.305  -11.305   -3.0745]
 [-11.305   -2.305   -2.305   -2.305   -3.0745]
 [ -2.305  -11.305   -2.305   -2.305  -10.9   ]
 [ -2.305   -2.305   -2.305   -1.9     -1.    ]
 [ -2.305   -3.0745  -3.0745 -10.       0.    ]]
1.4620500000000005
step  3
[[ -3.0745   -3.0745   -3.0745  -12.0745   -3.76705]
 [-12.0745   -3.0745   -3.0745   -3.0745   -3.76705]
 [ -3.0745  -12.0745   -3.0745   -2.71    -10.9    ]
 [ -3.0745   -3.0745   -2.71     -1.9      -1.     ]
 [ -3.76705  -3.76705  -3.439   -10.        0.     ]]
0.6925500000000007
step  4
[[ -3.76705   -3.76705   -3.76705  -12.76705   -4.390345]
 [-12.76705   -3.76705   -3.76705   -3.439     -4.0951  ]
 [ -3.76705  -12.76705   -3.439     -2.71     -10.9     ]
 [ -3.7

The final values for synchronous and asynchronous have come out the same, and it took the same amount of steps. This took me by surprise for a second until I considered the order values are calculated in, right to left and top to bottom, thus visiting the goal last. I'm going to reverse the order and see if there is a difference, which I do expect

In [13]:
delta = 1.0

#Values are intialized arbitrarily 
values = np.zeros((height, width))
values += -0.5
values[hole_locations] = hole_penalty
values[goal_location] = 0
old_values = np.copy(values)

asynchronous_step = 1
while delta > theta:
    for row in range(height-1, -1, -1):
        for col in range(width-1, -1, -1):
            if not goal_location[row][col]:
                next_state_vals = []
                for action in range(num_of_acts):
                    next_spot = actions[row][col][action]
                    if not (next_spot[0] == col and next_spot[1] == row):
                        next_val = rewards[row][col] + (gamma * values[next_spot[1]][next_spot[0]])
                        next_state_vals.append(next_val)
                if len(next_state_vals) != 0:
                    values[row][col] = max(next_state_vals)
    
    delta = np.amax(abs(np.subtract(old_values, values)))
    print(delta)
    print('step ', asynchronous_step)
    asynchronous_step += 1
    
    old_values = np.copy(values)

    print(values)

1.8049999999999997
step  1
[[ -2.305  -1.45   -1.45  -10.45   -2.305]
 [-10.45   -1.45   -1.45   -1.45   -1.45 ]
 [ -2.305 -10.45   -1.45   -1.45  -10.45 ]
 [ -1.45   -1.45   -1.45   -1.45   -1.   ]
 [ -1.45   -1.45   -1.45  -10.      0.   ]]
1.6245000000000012
step  2
[[ -3.76705  -3.0745   -2.305   -11.305    -3.0745 ]
 [-12.0745   -2.305    -2.305    -2.305    -2.305  ]
 [ -3.76705 -11.305    -2.305    -2.305   -10.9    ]
 [ -3.0745   -2.305    -2.305    -1.9      -1.     ]
 [ -2.305    -2.305    -2.305   -10.        0.     ]]
1.4620500000000005
step  3
[[ -4.9513105  -4.390345   -3.76705   -12.0745     -3.76705  ]
 [-13.390345   -3.76705    -3.0745     -3.0745     -3.0745   ]
 [ -4.68559   -12.0745     -3.0745     -2.71      -10.9      ]
 [ -4.0951     -3.439      -2.71       -1.9        -1.       ]
 [ -3.76705    -3.0745     -3.0745    -10.          0.       ]]
1.0206000000000004
step  4
[[ -5.6953279  -5.217031   -4.68559   -13.0951     -4.390345 ]
 [-14.217031   -4.68559    -4.0

There it is. Asynchronous value iteration has achieved the same thing as synchronous value iteration in two-thirds the about of steps, 6 compared to 9. However, it required being clever about how to implement the scanning. Out of curiosity, I will now implement asynchronous value iteration with pseudo-random ordering but being sure to visit every space once before moving to the next step. I expect this to be somewhere between the last two exercises.

In [14]:
delta = 1.0

#Values are intialized arbitrarily 
values = np.zeros((height, width))
values += -0.5
values[hole_locations] = hole_penalty
values[goal_location] = 0
old_values = np.copy(values)

asynchronous_step = 1

states = [(x,y) for x in range(width) for y in range(height) if not goal_location[y][x]]
random.shuffle(states)

while delta > theta:
    for state in states:
        col, row = state
        if not goal_location[row][col]:
            next_state_vals = []
            for action in range(num_of_acts):
                next_spot = actions[row][col][action]
                if not (next_spot[0] == col and next_spot[1] == row):
                    next_val = rewards[row][col] + (gamma * values[next_spot[1]][next_spot[0]])
                    next_state_vals.append(next_val)
            if len(next_state_vals) != 0:
                values[row][col] = max(next_state_vals)
    
    delta = np.amax(abs(np.subtract(old_values, values)))
    print(delta)
    print('step ', asynchronous_step)
    asynchronous_step += 1
    
    old_values = np.copy(values)
    
    random.shuffle(states)

    print(values)

1.8049999999999997
step  1
[[ -1.45   -1.45   -2.305 -11.305  -1.45 ]
 [-11.305  -1.45   -1.45   -1.45   -2.305]
 [ -1.45  -10.45   -1.45   -1.45  -10.9  ]
 [ -2.305  -1.45   -2.305  -1.45   -1.   ]
 [ -1.45   -1.45   -1.45  -10.      0.   ]]
1.6245000000000012
step  2
[[ -3.0745  -2.305   -3.0745 -11.305   -3.0745]
 [-11.305   -2.305   -2.305   -2.305   -3.0745]
 [ -3.0745 -12.0745  -2.305   -2.71   -10.9   ]
 [ -3.0745  -2.305   -2.305   -1.9     -1.    ]
 [ -2.305   -3.0745  -2.305  -10.       0.    ]]
1.7901000000000007
step  3
[[ -4.390345  -3.76705   -3.76705  -13.0951    -3.76705 ]
 [-12.76705   -3.0745    -3.0745    -3.439     -3.0745  ]
 [ -3.76705  -12.0745    -3.0745    -2.71     -10.9     ]
 [ -3.0745    -3.439     -2.71      -1.9       -1.      ]
 [ -3.76705   -3.76705   -3.0745   -10.         0.      ]]
1.0206000000000004
step  4
[[ -4.390345  -3.76705   -3.76705  -13.0951    -4.68559 ]
 [-12.76705   -3.76705   -4.0951    -3.439     -4.0951  ]
 [ -3.76705  -12.76705   -3.

I was correct, but I'm still surprised that it only took 7 steps. It makes sense that the previous two examples serve as upper and power bounds for efficiency. As such, one would expect randomness to fall in the middle. This might be worth keeping in mind for less intuitive optimal policies in the future.

I think that I've done enough with value iteration for now. I'm going to move on to policy iteration.

In [15]:
#Values are intialized arbitrarily 
values = np.zeros((height, width))
values += -0.5
values[hole_locations] = hole_penalty
values[goal_location] = 0
old_values = np.copy(values)

synchronous_step = 1

old_policy = np.zeros((height, width))
old_policy += terminal_state

total_policy_evaluations = 0
total_policy iterations = 0

while not np.array_equal(policy, old_policy):

    #policy evaluation
    while delta > theta:
        for row_2 in range(height):
            for col_2 in range(width):
                if not goal_location[row_2][col_2]:
                    values[row_2][col_2] = 0
                    for action in range(num_of_acts):
                        next_spot = actions[row_2][col_2][action]
                        values[row_2][col_2] += rewards[row_2][col_2] + (gamma * values[next_spot[1]][next_spot[0]])

        delta = np.amax(abs(np.subtract(old_values, values)))
        print(delta)
        print('step ', asynchronous_step)
        asynchronous_step += 1

        old_values = np.copy(values)

        print(values)
        
    #policy improvement
    for row_1 in range(height):
        for col_1 in range(width):
            if (not hole_locations[row_1][col_1]) and (not goal_location[row_1][col_1]):
                max_next_state_val = -100000 #this should never naturally occur
                for action in range(num_of_acts):
                    if (actions[row_1][col_1][action][0] != col or actions[row_1][col_1][action][1] != row):
                        next_spot = actions[row_1][col_1][action]
                        next_state_val = values[next_spot[1]][next_spot[0]]
                        if next_state_val > max_next_state_val:
                            max_next_state_val = next_state_val
                            policy[row_1][col_1] = action
            else:
                policy[row][col] = terminal_state

                
    total_policy_improvements += 1

    print('total policy improvements: ', total_policy_improvements)

    print(policy)
    
    synchronous_step += 1
    print('total cycles: ', synchronous_step)
    
print(policy)

IndentationError: unexpected indent (<ipython-input-15-bd2cf6d74763>, line 11)